In [1]:
import cobra
import json

In [4]:
def get_enzyme_constraint_model(json_model_file):
    """using enzyme concentration constraint
    json model to create a COBRApy model.

    Arguments
    ----------
    * json_model_file: json Model file.

    :return: Construct an enzyme-constrained model.
    """

    dictionary_model = json_load(json_model_file)
    model = cobra.io.json.load_json_model(json_model_file)
    prot_pool = cobra.Metabolite('prot_pool')
    prot_pool.compartment = 'c'
    coefficients = dict()
    for rxn in model.reactions:
        for eachr in dictionary_model['reactions']:
            if rxn.id == eachr['id']:
                if eachr['kcat_MW']:
                    rxn.add_metabolites({prot_pool:-1/float(eachr['kcat_MW'])})
                    # coefficients[rxn.forward_variable] = 1 / float(eachr['kcat_MW'])
                break

    lowerbound = dictionary_model['enzyme_constraint']['lowerbound']
    upperbound = dictionary_model['enzyme_constraint']['upperbound']
    
    R_pool = cobra.Reaction('enzyme_pool')
    R_pool.add_metabolites({prot_pool:1})
    R_pool.lower_bound = lowerbound
    R_pool.upper_bound = upperbound
    model.add_reactions([R_pool])
    #print(upperbound)
    # constraint = model.problem.Constraint(0, lb=lowerbound, ub=upperbound)
    # model.add_cons_vars(constraint)
    # model.solver.update()
    # constraint.set_linear_coefficients(coefficients=coefficients)
    return model
def get_enzyme_constraint_model_original(json_model_file):
    """using enzyme concentration constraint
    json model to create a COBRApy model.

    Arguments
    ----------
    * json_model_file: json Model file.

    :return: Construct an enzyme-constrained model.
    """

    dictionary_model = json_load(json_model_file)
    model = cobra.io.json.load_json_model(json_model_file)
    # prot_pool = cobra.Metabolite('prot_pool')
    # prot_pool.compartment = 'c'
    coefficients = dict()
    for rxn in model.reactions:
        for eachr in dictionary_model['reactions']:
            if rxn.id == eachr['id']:
                if eachr['kcat_MW']:
                    # rxn.add_metabolites({prot_pool:-1/float(eachr['kcat_MW'])})
                    coefficients[rxn.forward_variable] = 1 / float(eachr['kcat_MW'])
                break

    lowerbound = dictionary_model['enzyme_constraint']['lowerbound']
    upperbound = dictionary_model['enzyme_constraint']['upperbound']
    
    # R_pool = cobra.Reaction('enzyme_pool')
    # R_pool.add_metabolites({prot_pool:1})
    # R_pool.lower_bound = lowerbound
    # R_pool.upper_bound = upperbound
    # model.add_reactions([R_pool])
    #print(upperbound)
    constraint = model.problem.Constraint(0, lb=lowerbound, ub=upperbound)
    model.add_cons_vars(constraint)
    model.solver.update()
    constraint.set_linear_coefficients(coefficients=coefficients)
    return model

def json_load(path):
    """Loads the given JSON file and returns it as dictionary.

    Arguments
    ----------
    * path: The path of the JSON file
    """
    with open(path) as f:
        dictionary = json.load(f)
    return dictionary

In [10]:
model_fn = '../../../models/p_putida/eciJN1463_uniprot_AutoPACMEN.json'

In [18]:
# model_fn = '../../../models/p_putida/iJN1463_uniprot_irreversible.json'

In [11]:
model = get_enzyme_constraint_model(model_fn)

Malformed gene_reaction_rule 'PP_2193 and (PP_5306 and PP_5307 and PP_5308' for PP_2193 and (PP_5306 and PP_5307 and PP_5308
Traceback (most recent call last):
  File "/Users/snorre/miniconda3/envs/reframed/lib/python3.9/site-packages/cobra/core/gene.py", line 350, in from_string
    tree = ast_parse(escaped_str, "<string>", "eval")
  File "/Users/snorre/miniconda3/envs/reframed/lib/python3.9/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
  File "<string>", line 1
    PP_2193 and (PP_5306 and PP_5307 and PP_5308
                                               ^
SyntaxError: unexpected EOF while parsing

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/snorre/miniconda3/envs/reframed/lib/python3.9/site-packages/cobra/core/gene.py", line 365, in from_string
    tree = ast_parse(escaped_str, "<string>", "eval")
  File "/Users/snorre/miniconda3/envs/reframed/lib/python3.9/ast.py", line 50, in

In [12]:
model.reactions.EX_glc__D_e.lower_bound = -2
model.optimize()
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e_reverse,0.001591,0,0.00%
cl_e,EX_cl_e_reverse,0.001591,0,0.00%
cobalt2_e,EX_cobalt2_e_reverse,0.001144,0,0.00%
cu2_e,EX_cu2_e_reverse,0.00106,0,0.00%
fe2_e,EX_fe2_e_reverse,0.005528,0,0.00%
glc__D_e,EX_glc__D_e,2,6,25.00%
glc__D_e,EX_glc__D_e_reverse,6,6,75.00%
k_e,EX_k_e_reverse,0.05966,0,0.00%
mg2_e,EX_mg2_e_reverse,0.002651,0,0.00%
mn2_e,EX_mn2_e_reverse,0.00106,0,0.00%


In [17]:
# Merge split exchange reactions
for r in model.boundary:
    if r.id[-7:] == 'reverse':
        # print(r, r.upper_bound)
        r_forward = model.reactions.get_by_id(r.id[:-8])
        if r.upper_bound != 0:
            r_forward.lower_bound = -r.upper_bound
            # print(r_forward,r_forward.name, r_forward.bounds)
        r.remove_from_model()

In [19]:
model.id = 'eciJN1463_uniprot_AutoPACMEN'
model.name = 'eciJN1463_uniprot_AutoPACMEN'

In [20]:
for m in model.metabolites:
    if not m.compartment in ['e', 'c']:
        m.compartment = m.id[-1]

In [22]:
model.compartments = {'c': 'Cytoplasm', 'e': 'Extracellular'}

In [23]:
model.reactions.EX_glc__D_e.lower_bound = -2
model.optimize()
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0007883,0,0.00%
cl_e,EX_cl_e,0.0007883,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0005671,0,0.00%
cu2_e,EX_cu2_e,0.0005255,0,0.00%
fe2_e,EX_fe2_e,0.00274,0,0.00%
glc__D_e,EX_glc__D_e,2,6,100.00%
k_e,EX_k_e,0.02957,0,0.00%
mg2_e,EX_mg2_e,0.001314,0,0.00%
mn2_e,EX_mn2_e,0.0005255,0,0.00%
mobd_e,EX_mobd_e,0.0006087,0,0.00%


In [24]:
cobra.io.write_sbml_model(model, model_fn.replace('.json', '.xml'))

In [13]:
model2_fn = '../../../models/p_putida/iJN1463.xml'
model2 = cobra.io.read_sbml_model(model2_fn)

In [14]:
model2.solver = 'gurobi'

In [16]:
model2.reactions.EX_glc__D_e.lower_bound = -2
model2.optimize()
model2.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0008139,0,0.00%
cl_e,EX_cl_e,0.0008139,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0005855,0,0.00%
cu2_e,EX_cu2_e,0.0005425,0,0.00%
fe2_e,EX_fe2_e,0.002829,0,0.00%
glc__D_e,EX_glc__D_e,2,6,99.99%
k_e,EX_k_e,0.03053,0,0.00%
mg2_e,EX_mg2_e,0.001357,0,0.00%
mn2_e,EX_mn2_e,0.0005425,0,0.00%
mobd_e,EX_mobd_e,0.0006284,0,0.00%
